Title: DSC350 Week 6 Exercises  
Author: Stefanie Molin  
Date: 04 October 2024  
Modified By: Caleb Trimble 
Description: This program focuses on tailor dataframes based on specific values, creating bins for set values, creating crosstabs and pivot tables, conducting calculations on data, and merging dataframes. 
Codes in this program have been adapted from Hands-On Data Analysis with Pandas - Second Edition (Molin S., 2021)

In [2]:
# Problem 1

import numpy as np
import pandas as pd


df = pd.read_csv('earthquakes.csv')
df.head()

# Creates a DataFrame that shows the results for specific location, magnitude, and magnitude type.
japan_quakes = df[(df['parsed_place'] == "Japan") & (df['mag'] > 4.9) & (df['magType'] == "mb")]
print(japan_quakes)


      mag magType           time                       place  tsunami  \
2576  5.4      mb  1538697528010  37km E of Tomakomai, Japan        0   

     parsed_place  
2576        Japan  


In [5]:
# Problem 2

df = pd.read_csv('earthquakes.csv')
# Creates a DataFrame specifically for the magnitude type "ml".
ml_spec = df[(df['magType'] == "ml")] 
df.head()
# Creates a new DataFrame featuring 9 bins, with each bin representing a specific magnitude.
ml_binned = pd.cut(ml_spec.mag, bins=9, labels=['1', '2', '3', '4', '5', '6', '7', '8', '9'])
# Counts the number of values in each of the 5 bins. 
ml_binned.value_counts()

mag
4    2316
5    1651
3    1515
2     582
6     496
7     142
1      80
8      19
9       2
Name: count, dtype: int64

In [40]:
# Problem 3

import numpy as np


df = pd.read_csv('faang.csv')
# Sets the format for the date column to the datetime data type instead of an object.
df['date'] = pd.to_datetime(df['date'])
# Sets the DataFrame index to the date column.
df.set_index('date', inplace=True)

# Creates a new DataFrame that resamples the data to a 1-month time period and groups the data by the ticker column.
df_resample = df.groupby('ticker').resample('1M').agg({
# Calculates various statistics using numpy functions.
    'open': np.mean,  
    'high': np.max,
    'low': np.min,
    'close': np.mean,
    'volume': np.sum,
})
print(df_resample.head())


                        open       high        low      close        volume
ticker date                                                                
AAPL   2018-01-31  43.505357  45.025002  41.174999  43.501309  2.638718e+09
       2018-02-28  41.819079  45.154999  37.560001  41.909737  3.711577e+09
       2018-03-31  43.761786  45.875000  41.235001  43.624048  2.854911e+09
       2018-04-30  42.441310  44.735001  40.157501  42.458572  2.664617e+09
       2018-05-31  46.239091  47.592499  41.317501  46.384205  2.483905e+09


In [6]:
# Problem 4

import pandas as pd

tsunamis = pd.read_csv('earthquakes.csv')

# Calculates the maximum magnitude for each combination of tsunami and magType
max_mag = tsunamis.groupby(['tsunami', 'magType'])['mag'].max().reset_index()

# Creates the crosstab
crosstab = pd.crosstab(
    index=tsunamis['tsunami'],
    columns=tsunamis['magType'],
    values=tsunamis['mag'],
    aggfunc='max',
    margins=True,
    margins_name='Total Observations'
)

print(max_mag.head())
print(crosstab.head())


   tsunami magType   mag
0        0      mb  5.60
1        0   mb_lg  3.50
2        0      md  4.11
3        0      mh  1.10
4        0      ml  4.20
magType              mb  mb_lg    md   mh   ml  ms_20    mw  mwb  mwr  mww  \
tsunami                                                                      
0                   5.6    3.5  4.11  1.1  4.2    NaN  3.83  5.8  4.8  6.0   
1                   6.1    NaN   NaN  NaN  5.1    5.7  4.41  NaN  NaN  7.5   
Total Observations  6.1    3.5  4.11  1.1  5.1    5.7  4.41  5.8  4.8  7.5   

magType             Total Observations  
tsunami                                 
0                                  6.0  
1                                  7.5  
Total Observations                 7.5  


In [55]:
# Problem 5
# Calculates the rolling 60-day aggregations. 

import numpy as np


df = pd.read_csv('faang.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df_resample = df.groupby('ticker').resample('60D').agg({
    'open': np.mean,
    'high': np.max,
    'low': np.min,
    'close': np.mean,
    'volume': np.sum,
})
print(df_resample.head())

                        open       high        low      close        volume
ticker date                                                                
AAPL   2018-01-02  42.762143  45.154999  37.560001  42.800357  6.699319e+09
       2018-03-03  43.025183  45.875000  40.157501  42.980671  5.384782e+09
       2018-05-02  46.820000  48.549999  43.450001  46.867619  4.380126e+09
       2018-07-01  50.187857  55.872501  45.855000  50.302619  4.007506e+09
       2018-08-30  55.575793  58.367500  53.080002  55.531159  5.759600e+09


In [65]:
# Problem 6

import pandas as pd

df = pd.read_csv('faang.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Creates a pivot table using the ticker as the index, OHLCV as the values, and defines the aggregate function.
pivot_table = df.pivot_table(
    index='ticker',
    values=['open', 'high', 'low', 'close', 'volume'],
    aggfunc='mean'
)

print(pivot_table)


              close         high          low         open        volume
ticker                                                                  
AAPL      47.263357    47.748526    46.795877    47.277859  1.360803e+08
AMZN    1641.726176  1662.839839  1619.840519  1644.072709  5.648994e+06
FB       171.510956   173.613347   169.303148   171.472948  2.765860e+07
GOOG    1113.225134  1125.777606  1101.001658  1113.554101  1.741965e+06
NFLX     319.290319   325.219322   313.187330   319.620558  1.146962e+07


In [10]:
# Problem 7
import pandas as pd

df = pd.read_csv('faang.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Creates the DataFrame for z-scores, and defines the specific index to use (AMZN).
q4_AMZN_z_scores = df[(df['ticker'] == "AMZN")]\
    .loc['2018-Q4', ['high', 'low', 'open', 'close', 'volume']]\
    .apply(lambda x: x.sub(x.mean()).div(x.std()))
# Provides the description of the DataFrame.
q4_AMZN_z_scores.describe().T


,count,mean,std,min,25%,50%,75%,max
high,63.0,-1.656523e-16,1.0,-2.159820,-0.772677,-0.055727,0.635606,2.368006
low,63.0,-7.648203e-16,1.0,-2.187566,-0.729262,-0.083067,0.643712,2.502113
open,63.0,-6.379377e-16,1.0,-2.179582,-0.662691,-0.021091,0.664414,2.337813
close,63.0,6.696583e-17,1.0,-2.226185,-0.672697,-0.030371,0.696701,2.385848
volume,63.0,1.599250e-16,1.0,-1.928641,-0.662240,-0.282926,0.480970,2.926152


Line 10 - Defines what we want to see. In this case we're looking for quarter 4 of 2018, and we want the z-scores to be reflected for all numerical columns.  
Line 11 - Applies the z-score calculation using apply/lambda.

In [16]:
# Problem 8
import pandas as pd

# Creates the initial DataFrame
data = {
    'ticker': ['FB'] * 3,  # Sets the ticker values as 'FB' for the 3 rows.
    'date': ['2018-07-25', '2018-03-19', '2018-03-20'],
    'event': ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']
}
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
# Sets index to ['date', 'ticker']
df.set_index(['date', 'ticker'], inplace=True)

faang = pd.read_csv('faang.csv')

# Performs the outer merge
outer_join = pd.merge(
    df.reset_index(), faang.reset_index(), 
    on=['date', 'ticker'], how='outer', indicator=True
)

# Concatenates the result
result = pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0)
    for kind in outer_join._merge.unique()
]).sort_index()

print(result)


          date ticker                      event        high         low  \
1   2018-03-19     FB  Cambridge Analytica story  177.169998  170.059998   
2   2018-03-20     FB          FTC investigation  170.199997  161.949997   
8   2018-01-09     FB                        NaN  188.800003  187.100006   
302 2018-03-16   AAPL                        NaN   44.779999   44.404999   

           open       close       volume      _merge  
1    177.009995  172.559998   88140100.0        both  
2    167.470001  168.149994  129851800.0        both  
8    188.699997  187.869995   12393100.0  right_only  
302   44.662498   44.505001  157618800.0  right_only  


In [18]:
# Problem 9

import pandas as pd

faang = pd.read_csv('faang.csv')
faang['date'] = pd.to_datetime(faang['date'])
faang.set_index(['date', 'ticker'], inplace=True)

# Groups by ticker and applies the transformation using the transform() method.  
faang_transformed = faang.groupby('ticker').transform(lambda x: x / x.iloc[0])

print(faang_transformed)


                       high       low      open     close    volume
date       ticker                                                  
2018-01-02 FB      1.000000  1.000000  1.000000  1.000000  1.000000
2018-01-03 FB      1.017623  1.021290  1.023638  1.017914  0.930294
2018-01-04 FB      1.025498  1.036891  1.040635  1.016040  0.764708
2018-01-05 FB      1.029298  1.041566  1.044518  1.029931  0.747828
2018-01-08 FB      1.040313  1.049451  1.053579  1.037813  0.991340
...                     ...       ...       ...       ...       ...
2018-12-24 GOOG    0.940578  0.928131  0.928993  0.916638  1.284987
2018-12-26 GOOG    0.974750  0.940463  0.943406  0.976019  1.917663
2018-12-27 GOOG    0.978396  0.953857  0.970248  0.980169  1.704751
2018-12-28 GOOG    0.989334  0.988395  1.001221  0.973784  1.143180
2018-12-31 GOOG    0.986653  0.979296  1.002499  0.972404  1.206610

[1255 rows x 5 columns]
